<a href="https://colab.research.google.com/github/gmauricio-toledo/NLP-MCD/blob/main/08-TopicDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Topic Detection</h1>

<h2>Topic Modelling</h1>

En esta notebook ahondaremos un poco más en la tarea de *Topic Detection*. Para esto usaremos varias técnicas, algunas de ellas nuevas:

* Clustering en representaciones vectoriales de documentos.
* LDA. Implementación en [gensim](https://radimrehurek.com/gensim/models/ldamodel.html).
* LSA. Implementación en [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html).

Además, evaluaremos estas tareas usando métricas propias de la tarea.
* [Coherence](https://radimrehurek.com/gensim/models/coherencemodel.html): [source](https://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD

import spacy

from gensim import models, corpora
from gensim.models.coherencemodel import CoherenceModel

nltk.download('stopwords')

In [ ]:
!pip install wordcloud

Consideraremos un corpus de 92579 documentos de texto, consisten en noticias de CNN. **Origen desconocido**

In [ ]:
!gdown 1S-KYaCpb39vMphrkdnceXUkUhzHeapt7

In [ ]:
with open('cnn_articles.txt', 'r', encoding='utf8') as f:
    articles = f.read().split('@delimiter')

print(len(articles))

cnn_df = pd.DataFrame({'document':articles})
cnn_df

Nos quedamos solamente con los primeros 10000 artículos.

In [ ]:
corpus = articles[:10000]

Exploremos los documentos, aquí podemos ver algunas palabras que podemos añadir a la lista de stopwords.

In [ ]:
from wordcloud import WordCloud

wc = WordCloud(background_color="white", max_words=2000, contour_width=3, contour_color='steelblue')
wordcloud = wc.generate(" ".join(corpus))

plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

Hacemos el preprocesamiento usando un pipeline *ligero* de scipy. Tarda alrededor de 40s.

In [ ]:
%%time

nlp = spacy.blank('en')

tokenized_docs = [[t.text for t in tok_doc if
          not t.is_punct and \
          not t.is_space and \
          t.is_alpha] for tok_doc in nlp.pipe(corpus) ]

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['said'])

In [ ]:
tokenized_docs = [[w for w in doc if w not in stopwords] for doc in tokenized_docs]

In [ ]:
print(tokenized_docs[0])

🛑 Si usamos un pipeline más completo podemos tardar hasta 12 minutos.

In [ ]:
%%time

# nlp = spacy.load("en_core_web_sm", disable=["ner"])

# tokenized_docs = [[t.text for t in tok_doc if
#           not t.is_punct and \
#           not t.is_space and \
#           t.is_alpha] for tok_doc in nlp.pipe(corpus) ]

## LSA con scikit-learn

In [ ]:
docs = [" ".join(doc) for doc in tokenized_docs]

print(docs[:3])

vectorizer = TfidfVectorizer(stop_words='english',
                            max_features= 1000,
                            smooth_idf=True)

X = vectorizer.fit_transform(docs)

X.shape

In [ ]:
svd_model = TruncatedSVD(n_components=10, algorithm='randomized',
                         n_iter=100, random_state=122)

svd_model.fit(X)
len(svd_model.components_)

In [ ]:
svd_model.components_.shape

In [ ]:
terms = vectorizer.get_feature_names_out()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:10]
    print(f"Topic {str(i)}: ")
    for t in sorted_terms:
        print(t[0], end=', ')
    print()

In [ ]:
svd_model.transform(X).shape

In [ ]:
topics = np.argmax(svd_model.transform(X), axis=1)
num_topics = len(np.unique(topics))

docs_idxs_per_topic = [np.where(topics == i)[0] for i in range(num_topics)]

fig, axs = plt.subplots(1, num_topics, figsize=(15, 15))
for ax,j in zip(axs.flatten(),range(num_topics)):
    topic_docs = " ".join([docs[i] for i in docs_idxs_per_topic[j]])
    wc = WordCloud(background_color="white", max_words=2000).generate(topic_docs)
    ax.imshow(wc, interpolation='bilinear')
    ax.set_title(f"Topic {j}")
    ax.axis("off")
fig.tight_layout()
fig.show()

In [ ]:
import math

def get_umass_score(dt_matrix, i, j):
    zo_matrix = (dt_matrix > 0).astype(int)
    col_i, col_j = zo_matrix[:, i], zo_matrix[:, j]
    col_ij = col_i + col_j
    col_ij = (col_ij == 2).astype(int)
    Di, Dij = col_i.sum(), col_ij.sum()
    return math.log((Dij + 1) / Di)

def get_topic_coherence(dt_matrix, topic, n_top_words):
    indexed_topic = zip(topic, range(0, len(topic)))
    topic_top = sorted(indexed_topic, key=lambda x: 1 - x[0])[0:n_top_words]
    coherence = 0
    for j_index in range(0, len(topic_top)):
        for i_index in range(0, j_index - 1):
            i = topic_top[i_index][1]
            j = topic_top[j_index][1]
            coherence += get_umass_score(dt_matrix, i, j)
    return coherence

def get_average_topic_coherence(dt_matrix, topics, n_top_words):
    total_coherence = 0
    for i in range(0, len(topics)):
        total_coherence += get_topic_coherence(dt_matrix, topics[i], n_top_words)
    return total_coherence / len(topics)

In [ ]:
get_average_topic_coherence(X, svd_model.components_, 10)

In [ ]:
import matplotlib.pyplot as plt

num_topics = [3,5,7,10,15,20]
coherences = []

for k in num_topics:
    svd_model = TruncatedSVD(n_components=k, algorithm='randomized',
                         n_iter=100, random_state=122)
    svd_model.fit(X)
    coherences.append(get_average_topic_coherence(X, svd_model.components_, 10))

plt.figure()
plt.plot(num_topics, coherences)
plt.xlabel('Number of topics')
plt.ylabel('Coherence')
plt.xticks(num_topics)
plt.show()

## [LDA](https://radimrehurek.com/gensim/models/ldamodel.html) con gensim

Para usar la implementación de LDA de gensim necesitamos un diccionario relacionando los índices de las palabras y las palabras. Esta información ya la tenemos con el vectorizador TF-IDF.

El atributo `vocabulary_` de la clase `TfidfVectorizer` es un diccionario de la forma
            
        word: word_index

In [ ]:
vectorizer.vocabulary_

Para el modelo LDA de gensim necesitamos especificar un diccionario de la forma
            
        word_index: word

In [ ]:
dicc = dict(zip(vectorizer.vocabulary_.values(),vectorizer.vocabulary_.keys()))

Necesitamos también especificar una matriz sparse de scipy.

In [ ]:
from scipy.sparse import csr_matrix

X_csc = csr_matrix(X)

In [ ]:
%%time

from gensim.matutils import Sparse2Corpus

lda_model = models.LdaModel(corpus=Sparse2Corpus(X_csc,documents_columns=False), num_topics=10, id2word=dicc, random_state=1)

In [ ]:
lda_model.get_topics().shape

In [ ]:
word_id = 784

word_topics = lda_model.get_term_topics(word_id=word_id,minimum_probability=0)
word_topics = sorted(word_topics,key=lambda x: x[1], reverse=True)
print(f"Word: {dicc[word_id]}")
print(f"Topics: {word_topics}")

In [ ]:
lda_model.print_topics()

Podemos inspeccionar más a detalle un tópico

In [ ]:
lda_model.show_topic(topicid=9, topn=15)

In [ ]:
word = 'team'
word_id = vectorizer.vocabulary_[word]

word_topics = lda_model.get_term_topics(word_id=word_id,minimum_probability=0)
word_topics = sorted(word_topics,key=lambda x: x[1], reverse=True)
print(f"Word: {word}")
print(f"Topics: {word_topics}")

In [ ]:
lda_model.show_topics(num_topics=10, num_words=10, log=False, formatted=True)

In [ ]:
from gensim.corpora import Dictionary

dictionary = Dictionary.from_corpus(Sparse2Corpus(X_csc,documents_columns=False), id2word=dicc)
bow = dictionary.doc2bow(tokenized_docs[0])

topics = lda_model.get_document_topics(bow=bow, minimum_probability=None)

print(topics)

Veamos los tópicos de todos los documentos

In [ ]:
topics_list = []

for doc in tokenized_docs:
    bow = dictionary.doc2bow(doc)
    topics = lda_model.get_document_topics(bow=bow, minimum_probability=None)
    topics = sorted(topics,key=lambda x: x[1], reverse=True)
    topics_list.append(topics[0])

print(topics_list[:10])

In [ ]:
topics = np.array([x[0] for x in topics_list])
num_topics = len(np.unique(topics))

docs_idxs_per_topic = [np.where(topics == i)[0] for i in range(num_topics)]

fig, axs = plt.subplots(1, num_topics, figsize=(15, 15))
for ax,j in zip(axs.flatten(),range(num_topics)):
    topic_docs = " ".join([docs[i] for i in docs_idxs_per_topic[j]])
    wc = WordCloud(background_color="white", max_words=2000).generate(topic_docs)
    ax.imshow(wc, interpolation='bilinear')
    ax.set_title(f"Topic {j}")
    ax.axis("off")
fig.tight_layout()
fig.show()

Evaluación

La coherencia mide la distancia relativa entre palabras dentro de un tópico. Hay dos tipos principales:
* `c_v` típicamente está en $0 < x < 1$.
* `u_mass` típicamente es negativo.

Valores más altos son mejores.

The coherence of a topic is regarded as the sum of pairwise distributional similarity scores over the set of topic words, V:

$$\text{coh}(V) = \sum_{v_i,v_j \in V} \text{score}(v_i,v_j,ɛ).$$

* `uci` es extrínseca, los conteos se hacen en corpus externos.
$$\text{score}(v_i,v_j,ɛ) = \log\frac{p(w_i,w_j) + ɛ}{p(w_i)p(w_j)}$$

* `u_mass` en intrínseca, los conteos se hacen en corpus internos y no es simétrica.
$$\text{score}(v_i,v_j,ɛ) = \log\frac{D(w_i,w_j) + ɛ}{D(w_i)}$$

Referencias: [artículo original coherencia UMASS](https://aclanthology.org/D11-1024.pdf), [artículo comparando UMASS & UCI](https://aclanthology.org/D12-1087.pdf), [discusión en github](https://github.com/piskvorky/gensim/pull/710#issuecomment-425344644), [otra referencia](https://qpleple.com/topic-coherence-to-evaluate-topic-models/).

In [ ]:
cm = CoherenceModel(model=lda_model,
                    coherence='u_mass',
                    corpus=Sparse2Corpus(X_csc,documents_columns=False),
                    )
coherence = cm.get_coherence()
coherence

Si queremos usar las estrategias: `c_v`, `c_uci`, `c_npmi` tenemos que proporcionar información del corpus.

In [ ]:
from gensim.corpora import Dictionary

dictionary = Dictionary.from_corpus(Sparse2Corpus(X_csc,documents_columns=False), id2word=dicc)

cm = CoherenceModel(model=lda_model,
                    corpus=Sparse2Corpus(X_csc,documents_columns=False),
                    coherence='c_v',
                    texts=tokenized_docs,
                    dictionary=dictionary)
coherence = cm.get_coherence()
coherence

También podemos ver la coherencia por tópico:

In [ ]:
cm.get_coherence_per_topic()

Podemos decidir el número de tópicos en función del valor de coherencia

In [ ]:
num_topics = [3+k for k in range(15)]
coherences = []

for k in num_topics:
    lda_model = models.LdaModel(corpus=Sparse2Corpus(X_csc,documents_columns=False),
                                num_topics=k, id2word=dicc, random_state=1)
    cm = CoherenceModel(model=lda_model,
                    coherence='u_mass',
                    corpus=Sparse2Corpus(X_csc,documents_columns=False),
                    )
    coherences.append(cm.get_coherence())

In [ ]:
plt.figure()
plt.plot(num_topics, coherences)
plt.xlabel('Number of topics')
plt.ylabel('Coherence')
plt.xticks(num_topics)
plt.show()

# 20newsgroups

Podemos repetir el experimento con el corpus `20newsgroups`

In [ ]:
train_data = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
test_data = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

X_train = train_data.data
X_test = test_data.data

y_train = train_data.target
y_test = test_data.target

In [ ]:
news_df = pd.DataFrame({'document':X_train,
                        'topic': y_train})
news_df

# 🟥 Tarea

## Introducción

Si tenemos un etiquetado ground truth podemos usar métricas que comparan entre agrupamientos. Algunas de estas métricas suelen usarse en tareas de clustering. Algunos ejemplos son:

1. [Rand Index **RI**](https://scikit-learn.org/stable/modules/clustering.html#rand-index)
2. [Mutual Information based scores **MI**](https://scikit-learn.org/stable/modules/clustering.html#mutual-information-based-scores)
3. [Homogeneity, completeness and V-measure **HCV**](https://scikit-learn.org/stable/modules/clustering.html#homogeneity-completeness-and-v-measure)

## Ejercicios

Vamos a realizar la tarea de topic modeling usando el corpus `20newsgroups`

1. Usando LDA obten 20 tópicos, mide el desempeño usando una métrica de cada uno de los 3 grupos descritos anteriormentes (RI, MI, HCV). También mide el desempeño usando la coherencia.
2. Usando LSA obten 20 tópicos, mide el desempeño usando una métrica de cada uno de los 3 grupos descritos anteriormentes (RI, MI, HCV).
3. Usando un algoritmo de clustering donde se especifique el número de clusters, obtener 20 clusters que, idealmente, representen los tópicos. El algoritmo de clustering lo aplicaras a las representaciones BOW o TF-IDF. Escoge la que mejor desempeño tenga de acuerdo a alguna de las métricas de los 3 grupos anteriores (RI, MI, HCV).

4. En cada una de las 3 estrategias haz una exploración manual de algunos documentos, ¿parece haber coherencia?
5. En cada una de las 3 estrategias haz una nube de palabras por cada tópico, ¿parece haber coherencia en el vocabulario?

## Conclusiones

Redacta un pequeño texto respondiendo las siguientes preguntas.

* ¿Cuál método produjo mejores resultados en este caso?
* El hecho de tener las etiquetas reales de tópicos, ¿facilita la tarea?